## Importing Libraries and Data

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import string

In [2]:
data_path = r"F:\Machine Learning and AI\Deep Learning with Tensorflow IBM\Projects\Text Auto Generation\t8.shakespeare.txt"

In [3]:
text_data = open(data_path, 'rb').read().decode(encoding = 'utf-8')

## Data Preprocessing

In [4]:
print(text_data[10460])

In [5]:
text_data = text_data[10460:]

### Unique Character Count and Mapping them to integers

In [6]:
char = sorted(set(text_data))

In [10]:
char[6]

'('

In [7]:
len(char)

84

In [8]:
char_index = {f: i for i, f in enumerate(char)}
index_char = np.array(char)

In [9]:
text_to_int = np.array([char_index[c] for c in text_data])

### Text Preprocessing

In [10]:
seq_len = 50
char_per_epoch = len(text_data)//(seq_len + 1)

In [11]:
char_data = tf.data.Dataset.from_tensor_slices(text_to_int)

In [12]:
data_seq = char_data.batch(seq_len + 1, drop_remainder = True)

In [13]:
def x_y_split(portion):
    x = portion[: -1]
    y = portion[1 :]
    return x, y
data_final = data_seq.map(x_y_split)

In [14]:
batch_size = 64
buffer = 10000

In [15]:
data_final = data_final.shuffle(buffer).batch(batch_size, drop_remainder = True)

## Model Building

In [24]:
model_rnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(char), 16, batch_input_shape = [batch_size, None]),
    tf.keras.layers.GRU(32, return_sequences = True, stateful = True, recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.GRU(64, return_sequences = True, stateful = True, recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(len(char))
])

In [25]:
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 16)            1344      
_________________________________________________________________
gru_4 (GRU)                  (64, None, 32)            4800      
_________________________________________________________________
gru_5 (GRU)                  (64, None, 64)            18816     
_________________________________________________________________
dense_2 (Dense)              (64, None, 84)            5460      
Total params: 30,420
Trainable params: 30,420
Non-trainable params: 0
_________________________________________________________________


In [26]:
model_rnn.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [29]:
rnn_fit = model_rnn.fit(data_final, epochs = 50)

Epoch 1/50
1669/1669 [==============================] - 110s 66ms/step - loss: 2.9707 - accuracy: 0.2443
Epoch 2/50
1669/1669 [==============================] - 102s 60ms/step - loss: 2.7798 - accuracy: 0.2804
Epoch 3/50
1669/1669 [==============================] - 100s 60ms/step - loss: 2.6892 - accuracy: 0.2608
Epoch 4/50
1669/1669 [==============================] - 99s 59ms/step - loss: 2.6445 - accuracy: 0.2893
Epoch 5/50
1669/1669 [==============================] - 99s 59ms/step - loss: 2.6855 - accuracy: 0.25720s - loss: 2
Epoch 6/50
1669/1669 [==============================] - 101s 60ms/step - loss: 2.5590 - accuracy: 0.3220
Epoch 7/50
1669/1669 [==============================] - 99s 59ms/step - loss: 2.6585 - accuracy: 0.3099
Epoch 8/50
1669/1669 [==============================] - 101s 60ms/step - loss: 2.6351 - accuracy: 0.3220s - loss: 2
Epoch 9/50
1669/1669 [==============================] - 113s 67ms/step - loss: 2.5734 - accuracy: 0.3292
Epoch 10/50
1669/1669 [============

In [30]:
model_rnn.save('text_rnn.h5')